#04_DSMarket_SS_calculation

## 1. Import

In [106]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error
from scipy.stats import norm

In [107]:
df_sales = pd.read_parquet("df_forecasting.parquet", engine="pyarrow")
print(df_sales.shape)
df_sales.head(3)

(8354260, 15)


,id,Unnamed: 0,item,category,department,store,store_code,region,yearweek,n_sales,revenue,avg_sell_price,event,cluster_store,cluster_item
0,ACCESORIES_1_001_BOS_1,0,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,South_End,BOS_1,Boston,201104,0,0.0,NaN,Without event,1,0
1,ACCESORIES_1_001_BOS_1,1,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,South_End,BOS_1,Boston,201105,0,0.0,NaN,SuperBowl,1,0
2,ACCESORIES_1_001_BOS_1,2,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,South_End,BOS_1,Boston,201106,0,0.0,NaN,Without event,1,0


In [108]:
df_rmse = pd.read_parquet("rmse_por_id.parquet", engine="pyarrow")
print(df_rmse.shape)
df_rmse.head(3)

(30490, 2)


,id,rmse_mean
0,ACCESORIES_1_001_BOS_1,1.006209
1,ACCESORIES_1_001_BOS_2,1.031386
2,ACCESORIES_1_001_BOS_3,1.477416


In [109]:
df_rmse.head(3)

,id,rmse_mean
0,ACCESORIES_1_001_BOS_1,1.006209
1,ACCESORIES_1_001_BOS_2,1.031386
2,ACCESORIES_1_001_BOS_3,1.477416


In [110]:
df_sales.head(3)

,id,Unnamed: 0,item,category,department,store,store_code,region,yearweek,n_sales,revenue,avg_sell_price,event,cluster_store,cluster_item
0,ACCESORIES_1_001_BOS_1,0,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,South_End,BOS_1,Boston,201104,0,0.0,NaN,Without event,1,0
1,ACCESORIES_1_001_BOS_1,1,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,South_End,BOS_1,Boston,201105,0,0.0,NaN,SuperBowl,1,0
2,ACCESORIES_1_001_BOS_1,2,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,South_End,BOS_1,Boston,201106,0,0.0,NaN,Without event,1,0


In [111]:
df_prediction = pd.read_csv("df_prediction_4.csv")  # <-- cambia qui
print(df_prediction.shape)
df_prediction.head(3)


(8476220, 16)


,Unnamed: 0,id,yearweek,avg_sell_price,event,cluster_store,cluster_item,n_sales,date,revenue,item,category,department,store,store_code,region
0,0,ACCESORIES_1_001_BOS_1,201104,NaN,0,2,0,0,2011-01-24,0.0,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,South_End,BOS_1,Boston
1,1,ACCESORIES_1_001_BOS_1,201105,NaN,1,2,0,0,2011-01-31,0.0,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,South_End,BOS_1,Boston
2,2,ACCESORIES_1_001_BOS_1,201106,NaN,0,2,0,0,2011-02-07,0.0,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,South_End,BOS_1,Boston


In [112]:
df_prediction_sorted = pd.Series(df_prediction["yearweek"].unique()).sort_values()

In [113]:
last_4 = df_prediction_sorted.tail(4)

In [114]:
last_4

274    201617
275    201618
276    201619
277    201620
dtype: int64

In [115]:
df_prediction = df_prediction[df_prediction["yearweek"].isin(last_4)]

In [116]:
df_prediction

,Unnamed: 0,id,yearweek,avg_sell_price,event,cluster_store,cluster_item,n_sales,date,revenue,item,category,department,store,store_code,region
8354260,8354260,ACCESORIES_1_001_BOS_1,201617,10.9858,0,2,0,3,2016-04-25,32.9574,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,South_End,BOS_1,Boston
8354261,8354261,ACCESORIES_1_001_BOS_1,201618,10.9858,0,2,0,3,2016-05-02,32.9574,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,South_End,BOS_1,Boston
8354262,8354262,ACCESORIES_1_001_BOS_1,201619,10.9858,0,2,0,3,2016-05-09,32.9574,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,South_End,BOS_1,Boston
8354263,8354263,ACCESORIES_1_001_BOS_1,201620,10.9858,0,2,0,2,2016-05-16,21.9716,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,South_End,BOS_1,Boston
8354264,8354264,ACCESORIES_1_001_BOS_2,201617,10.9858,0,1,0,2,2016-04-25,21.9716,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,Roxbury,BOS_2,Boston
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8476215,8476215,SUPERMARKET_3_827_PHI_2,201620,1.2000,0,0,0,14,2016-05-16,16.8000,SUPERMARKET_3_827,SUPERMARKET,SUPERMARKET_3,Yorktown,PHI_2,Philadelphia
8476216,8476216,SUPERMARKET_3_827_PHI_3,201617,1.2000,0,2,0,11,2016-04-25,13.2000,SUPERMARKET_3_827,SUPERMARKET,SUPERMARKET_3,Queen_Village,PHI_3,Philadelphia
8476217,8476217,SUPERMARKET_3_827_PHI_3,201618,1.2000,0,2,0,8,2016-05-02,9.6000,SUPERMARKET_3_827,SUPERMARKET,SUPERMARKET_3,Queen_Village,PHI_3,Philadelphia
8476218,8476218,SUPERMARKET_3_827_PHI_3,201619,1.2000,0,2,0,7,2016-05-09,8.4000,SUPERMARKET_3_827,SUPERMARKET,SUPERMARKET_3,Queen_Village,PHI_3,Philadelphia


In [117]:
df_prediction_mean = df_prediction.groupby("id")["n_sales"].mean().reset_index()
df_prediction_mean = df_prediction_mean.rename(columns={"n_sales": "DL"})

In [118]:
df_prediction_mean

,id,DL
0,ACCESORIES_1_001_BOS_1,2.75
1,ACCESORIES_1_001_BOS_2,1.75
2,ACCESORIES_1_001_BOS_3,2.75
3,ACCESORIES_1_001_NYC_1,6.75
4,ACCESORIES_1_001_NYC_2,4.75
...,...,...
30485,SUPERMARKET_3_827_NYC_3,31.25
30486,SUPERMARKET_3_827_NYC_4,10.00
30487,SUPERMARKET_3_827_PHI_1,14.00
30488,SUPERMARKET_3_827_PHI_2,15.00


In [119]:
df_rmse = df_rmse.merge(
    df_prediction_mean[['id', 'DL']],  
    on='id',                           
    how='left'                          
)

In [120]:
df_rmse

,id,rmse_mean,DL
0,ACCESORIES_1_001_BOS_1,1.006209,2.75
1,ACCESORIES_1_001_BOS_2,1.031386,1.75
2,ACCESORIES_1_001_BOS_3,1.477416,2.75
3,ACCESORIES_1_001_NYC_1,2.073110,6.75
4,ACCESORIES_1_001_NYC_2,2.703180,4.75
...,...,...,...
30485,SUPERMARKET_3_827_NYC_3,5.298480,31.25
30486,SUPERMARKET_3_827_NYC_4,4.934147,10.00
30487,SUPERMARKET_3_827_PHI_1,9.285024,14.00
30488,SUPERMARKET_3_827_PHI_2,7.789934,15.00


## 2. Cálculo del stock de seguridad

Para calcular el **stock de seguridad**, utilizaremos el **RMSE** obtenido para cada item y tienda durante la fase de predicción.
A partir de este valor, calcularemos el stock de seguridad del producto en cada tienda aplicando la siguiente fórmula:

**SS** = Z ×RMSE ×√L
​
- **Z** es el nivel de servicio deseado, que representa la probabilidad de no tener una rotura de stock durante el tiempo de reposición.
Cuanto mayor sea el nivel de servicio (por ejemplo, 95% o 99%), mayor será el valor de Z y, por tanto, el nivel de protección frente a la variabilidad de la demanda.

- **L** es el lead time o tiempo de aprovisionamiento, es decir, el número de periodos (en este caso semanas) que transcurren desde que se realiza el pedido hasta que la mercancía llega al almacén o tienda.

De esta forma, el stock de seguridad actúa como una reserva adicional que permite absorber la incertidumbre de la demanda y garantizar la disponibilidad del producto incluso en situaciones imprevistas.

In [121]:
service_level = 0.95
z = norm.ppf(service_level)
L = 1 #lead time of one week

In [122]:
z

np.float64(1.6448536269514722)

In [123]:
df_rmse["SS"] = z*df_rmse["rmse_mean"] * np.sqrt(L)

In [124]:
df_rmse

,id,rmse_mean,DL,SS
0,ACCESORIES_1_001_BOS_1,1.006209,2.75,1.655066
1,ACCESORIES_1_001_BOS_2,1.031386,1.75,1.696480
2,ACCESORIES_1_001_BOS_3,1.477416,2.75,2.430132
3,ACCESORIES_1_001_NYC_1,2.073110,6.75,3.409963
4,ACCESORIES_1_001_NYC_2,2.703180,4.75,4.446336
...,...,...,...,...
30485,SUPERMARKET_3_827_NYC_3,5.298480,31.25,8.715224
30486,SUPERMARKET_3_827_NYC_4,4.934147,10.00,8.115950
30487,SUPERMARKET_3_827_PHI_1,9.285024,14.00,15.272505
30488,SUPERMARKET_3_827_PHI_2,7.789934,15.00,12.813302


## 2. ROP — nivel de pedido (Reorder Point)

El punto de pedido (ROP) indica el nivel de inventario en el que se debe realizar un nuevo pedido para evitar una rotura de stock, y se calcula de la siguiente manera:

𝑅𝑂𝑃 =𝐷𝐿 +𝑆𝑆

donde:

**DL** = demanda media durante el lead time, que en nuestro caso utilizaremos como la demanda media del artículo en la tienda

**SS** = stock de seguridad

In [125]:
df_sales["year"] = df_sales["yearweek"].astype(str).str[:4].astype(int)

In [126]:
df_sales["n_sales_sum"] = df_sales.groupby("id")["n_sales"].transform("sum")

In [127]:
df_sales["n_sales_sum_2015"] = df_sales[df_sales["year"] == 2015].groupby("id")["n_sales"].transform("sum")

In [128]:
df_sales = df_sales.dropna(subset=["n_sales_sum_2015"]) #Eliminar las filas nulas que se generan

In [129]:
df_sales["revenue"] = df_sales["n_sales"] * df_sales["avg_sell_price"]

In [130]:
df_sales["revenue_sum"] = df_sales.groupby("id")["revenue"].transform("sum")

In [131]:
df_sales["avg_price"] = df_sales["revenue_sum"] / df_sales["n_sales_sum"]

In [132]:
columnas = ["id","item","store_code","category","avg_price","n_sales_sum_2015"]

In [133]:
df_sales = df_sales[columnas]

In [134]:
df_sales.shape

(1615970, 6)

In [135]:
df_sales = df_sales.drop_duplicates()

In [136]:
df_sales.shape

(30490, 6)

In [137]:
df_sales.columns

Index(['id', 'item', 'store_code', 'category', 'avg_price',
       'n_sales_sum_2015'],
      dtype='object')

In [138]:
df_final = df_sales.merge(
    df_rmse[['id','rmse_mean','SS','DL']],  # prendi solo le colonne necessarie
    on=['id'],
    how='right'  # mantiene tutte le righe di df_rmse
)

In [139]:
df_final

,id,item,store_code,category,avg_price,n_sales_sum_2015,rmse_mean,SS,DL
0,ACCESORIES_1_001_BOS_1,ACCESORIES_1_001,BOS_1,ACCESORIES,3.642665,87.0,1.006209,1.655066,2.75
1,ACCESORIES_1_001_BOS_2,ACCESORIES_1_001,BOS_2,ACCESORIES,5.000999,183.0,1.031386,1.696480,1.75
2,ACCESORIES_1_001_BOS_3,ACCESORIES_1_001,BOS_3,ACCESORIES,5.421564,190.0,1.477416,2.430132,2.75
3,ACCESORIES_1_001_NYC_1,ACCESORIES_1_001,NYC_1,ACCESORIES,4.119675,225.0,2.073110,3.409963,6.75
4,ACCESORIES_1_001_NYC_2,ACCESORIES_1_001,NYC_2,ACCESORIES,3.648421,180.0,2.703180,4.446336,4.75
...,...,...,...,...,...,...,...,...,...
30485,SUPERMARKET_3_827_NYC_3,SUPERMARKET_3_827,NYC_3,SUPERMARKET,0.397978,853.0,5.298480,8.715224,31.25
30486,SUPERMARKET_3_827_NYC_4,SUPERMARKET_3_827,NYC_4,SUPERMARKET,0.012903,1.0,4.934147,8.115950,10.00
30487,SUPERMARKET_3_827_PHI_1,SUPERMARKET_3_827,PHI_1,SUPERMARKET,0.575842,1311.0,9.285024,15.272505,14.00
30488,SUPERMARKET_3_827_PHI_2,SUPERMARKET_3_827,PHI_2,SUPERMARKET,0.636678,460.0,7.789934,12.813302,15.00


In [140]:
df_final["ROP"] = (df_final["DL"] + df_final["SS"]).round(0)

In [141]:
df_final["SS"] = df_final["SS"].round(0)

In [142]:
df_final

,id,item,store_code,category,avg_price,n_sales_sum_2015,rmse_mean,SS,DL,ROP
0,ACCESORIES_1_001_BOS_1,ACCESORIES_1_001,BOS_1,ACCESORIES,3.642665,87.0,1.006209,2.0,2.75,4.0
1,ACCESORIES_1_001_BOS_2,ACCESORIES_1_001,BOS_2,ACCESORIES,5.000999,183.0,1.031386,2.0,1.75,3.0
2,ACCESORIES_1_001_BOS_3,ACCESORIES_1_001,BOS_3,ACCESORIES,5.421564,190.0,1.477416,2.0,2.75,5.0
3,ACCESORIES_1_001_NYC_1,ACCESORIES_1_001,NYC_1,ACCESORIES,4.119675,225.0,2.073110,3.0,6.75,10.0
4,ACCESORIES_1_001_NYC_2,ACCESORIES_1_001,NYC_2,ACCESORIES,3.648421,180.0,2.703180,4.0,4.75,9.0
...,...,...,...,...,...,...,...,...,...,...
30485,SUPERMARKET_3_827_NYC_3,SUPERMARKET_3_827,NYC_3,SUPERMARKET,0.397978,853.0,5.298480,9.0,31.25,40.0
30486,SUPERMARKET_3_827_NYC_4,SUPERMARKET_3_827,NYC_4,SUPERMARKET,0.012903,1.0,4.934147,8.0,10.00,18.0
30487,SUPERMARKET_3_827_PHI_1,SUPERMARKET_3_827,PHI_1,SUPERMARKET,0.575842,1311.0,9.285024,15.0,14.00,29.0
30488,SUPERMARKET_3_827_PHI_2,SUPERMARKET_3_827,PHI_2,SUPERMARKET,0.636678,460.0,7.789934,13.0,15.00,28.0


## 3. EOQ (Economic Order Quantity)

El **EOQ** (Economic Order Quantity) o Cantidad Económica de Pedido es un modelo de gestión de inventarios que ayuda a las empresas a decidir cuántas unidades deben pedir cada vez para mantener los costos lo más bajos posible.

El objetivo es encontrar un equilibrio entre:

- Pedir demasiado a menudo, lo que aumenta los costos de pedido, y

- Pedir grandes cantidades, lo que incrementa los costos de almacenamiento.

En resumen, el EOQ permite **optimizar la gestión de inventarios**, garantizando que haya suficiente stock para satisfacer la demanda sin generar costos innecesarios

La fórmula clásica del EOQ es:

𝐸𝑂𝑄 = √(2𝐷⋅SS/𝐻)

donde:

**D** = demanda anual (unidades por año)

**S** = costo por pedido (costo de preparación o de orden)

**H** = costo de mantenimiento por unidad al año (holding cost per unit per year)

Esta fórmula determina la cantidad óptima de pedido que minimiza el costo total de inventario, es decir, la suma de los costos de pedido y los costos de mantenimiento.

Dado que ni **H** ni **S** están disponibles para este problema, vamos a **suponer sus valores en función de la categoría de los artículos**.
En particular, **H**, el costo de mantenimiento en almacén, se calculará como un **porcentaje del precio medio de venta** del producto, de la siguiente manera:

* **SUPERMARKET:** 15 % → productos que se venden más rápido, con mayor rotación y menos tiempo en almacén.
* **HOME & GARDEN:** 20 % → productos con una rotación menor.
* **ACCESORIES:** 25 % → productos que se venden con menos frecuencia, por lo tanto, con un mayor porcentaje de costo en caso de pedido.


In [143]:
h_dict = {
    'SUPERMARKET': 0.15,
    'HOME_&_GARDEN': 0.20,
    'ACCESORIES': 0.25
}

In [144]:
df_final['H_%'] = df_final['category'].map(h_dict)

In [145]:
df_final[df_final['category']=="ACCESORIES"].head()

,id,item,store_code,category,avg_price,n_sales_sum_2015,rmse_mean,SS,DL,ROP,H_%
0,ACCESORIES_1_001_BOS_1,ACCESORIES_1_001,BOS_1,ACCESORIES,3.642665,87.0,1.006209,2.0,2.75,4.0,0.25
1,ACCESORIES_1_001_BOS_2,ACCESORIES_1_001,BOS_2,ACCESORIES,5.000999,183.0,1.031386,2.0,1.75,3.0,0.25
2,ACCESORIES_1_001_BOS_3,ACCESORIES_1_001,BOS_3,ACCESORIES,5.421564,190.0,1.477416,2.0,2.75,5.0,0.25
3,ACCESORIES_1_001_NYC_1,ACCESORIES_1_001,NYC_1,ACCESORIES,4.119675,225.0,2.073110,3.0,6.75,10.0,0.25
4,ACCESORIES_1_001_NYC_2,ACCESORIES_1_001,NYC_2,ACCESORIES,3.648421,180.0,2.703180,4.0,4.75,9.0,0.25


In [146]:
df_final['H']=df_final['H_%']*df_final['avg_price']

In [147]:
mean_H = df_final["H"].mean()
mean_H

np.float64(0.30803362477529217)

In [148]:
df_final.loc[df_final['H'] == 0, 'H'] = mean_H

In [149]:
df_final

,id,item,store_code,category,avg_price,n_sales_sum_2015,rmse_mean,SS,DL,ROP,H_%,H
0,ACCESORIES_1_001_BOS_1,ACCESORIES_1_001,BOS_1,ACCESORIES,3.642665,87.0,1.006209,2.0,2.75,4.0,0.25,0.910666
1,ACCESORIES_1_001_BOS_2,ACCESORIES_1_001,BOS_2,ACCESORIES,5.000999,183.0,1.031386,2.0,1.75,3.0,0.25,1.250250
2,ACCESORIES_1_001_BOS_3,ACCESORIES_1_001,BOS_3,ACCESORIES,5.421564,190.0,1.477416,2.0,2.75,5.0,0.25,1.355391
3,ACCESORIES_1_001_NYC_1,ACCESORIES_1_001,NYC_1,ACCESORIES,4.119675,225.0,2.073110,3.0,6.75,10.0,0.25,1.029919
4,ACCESORIES_1_001_NYC_2,ACCESORIES_1_001,NYC_2,ACCESORIES,3.648421,180.0,2.703180,4.0,4.75,9.0,0.25,0.912105
...,...,...,...,...,...,...,...,...,...,...,...,...
30485,SUPERMARKET_3_827_NYC_3,SUPERMARKET_3_827,NYC_3,SUPERMARKET,0.397978,853.0,5.298480,9.0,31.25,40.0,0.15,0.059697
30486,SUPERMARKET_3_827_NYC_4,SUPERMARKET_3_827,NYC_4,SUPERMARKET,0.012903,1.0,4.934147,8.0,10.00,18.0,0.15,0.001935
30487,SUPERMARKET_3_827_PHI_1,SUPERMARKET_3_827,PHI_1,SUPERMARKET,0.575842,1311.0,9.285024,15.0,14.00,29.0,0.15,0.086376
30488,SUPERMARKET_3_827_PHI_2,SUPERMARKET_3_827,PHI_2,SUPERMARKET,0.636678,460.0,7.789934,13.0,15.00,28.0,0.15,0.095502


https://www.ascendsoftware.com/blog/the-average-cost-of-processing-a-purchase-order-a-detailed-analysis?utm_source=chatgpt.com

https://it.scribd.com/document/459659976/2019-HCKT-Metrics-Procurement-Cost-v1?utm_source=chatgpt.com

Dado que el costo por pedido (S) no está disponible, supondremos tres escenarios posibles aplicables a todas las categorías de productos:

Escenario mínimo: 10 € por pedido → procesos muy automatizados o simples.

Escenario medio: 25 € por pedido → nivel medio de digitalización y complejidad.

Escenario alto: 40 € por pedido → procesos manuales o con mayor carga administrativa y de control.

In [150]:
df_final["S"] = 20

In [151]:
df_final

,id,item,store_code,category,avg_price,n_sales_sum_2015,rmse_mean,SS,DL,ROP,H_%,H,S
0,ACCESORIES_1_001_BOS_1,ACCESORIES_1_001,BOS_1,ACCESORIES,3.642665,87.0,1.006209,2.0,2.75,4.0,0.25,0.910666,20
1,ACCESORIES_1_001_BOS_2,ACCESORIES_1_001,BOS_2,ACCESORIES,5.000999,183.0,1.031386,2.0,1.75,3.0,0.25,1.250250,20
2,ACCESORIES_1_001_BOS_3,ACCESORIES_1_001,BOS_3,ACCESORIES,5.421564,190.0,1.477416,2.0,2.75,5.0,0.25,1.355391,20
3,ACCESORIES_1_001_NYC_1,ACCESORIES_1_001,NYC_1,ACCESORIES,4.119675,225.0,2.073110,3.0,6.75,10.0,0.25,1.029919,20
4,ACCESORIES_1_001_NYC_2,ACCESORIES_1_001,NYC_2,ACCESORIES,3.648421,180.0,2.703180,4.0,4.75,9.0,0.25,0.912105,20
...,...,...,...,...,...,...,...,...,...,...,...,...,...
30485,SUPERMARKET_3_827_NYC_3,SUPERMARKET_3_827,NYC_3,SUPERMARKET,0.397978,853.0,5.298480,9.0,31.25,40.0,0.15,0.059697,20
30486,SUPERMARKET_3_827_NYC_4,SUPERMARKET_3_827,NYC_4,SUPERMARKET,0.012903,1.0,4.934147,8.0,10.00,18.0,0.15,0.001935,20
30487,SUPERMARKET_3_827_PHI_1,SUPERMARKET_3_827,PHI_1,SUPERMARKET,0.575842,1311.0,9.285024,15.0,14.00,29.0,0.15,0.086376,20
30488,SUPERMARKET_3_827_PHI_2,SUPERMARKET_3_827,PHI_2,SUPERMARKET,0.636678,460.0,7.789934,13.0,15.00,28.0,0.15,0.095502,20


In [152]:
df_final['EOQ'] = np.round(np.sqrt((2 * df_final['DL'] * df_final['S']) / df_final['H']))

In [154]:
filtered_df = df_final[
    (df_final['EOQ'].isin([0, np.inf])) 
]

In [155]:
mean_H = df_final["H"].mean()
mean_H

np.float64(0.3095995549766752)

In [156]:
df_final[df_final["DL"]>50]

,id,item,store_code,category,avg_price,n_sales_sum_2015,rmse_mean,SS,DL,ROP,H_%,H,S,EOQ
73,ACCESORIES_1_008_NYC_1,ACCESORIES_1_008,NYC_1,ACCESORIES,0.158750,3480.0,11.551718,19.0,73.75,93.0,0.25,0.039687,20,273.0
183,ACCESORIES_1_019_NYC_1,ACCESORIES_1_019,NYC_1,ACCESORIES,0.182732,2626.0,12.303301,20.0,55.00,75.0,0.25,0.045683,20,219.0
473,ACCESORIES_1_048_NYC_1,ACCESORIES_1_048,NYC_1,ACCESORIES,0.197485,4336.0,14.942349,25.0,103.75,128.0,0.25,0.049371,20,290.0
1532,ACCESORIES_1_158_BOS_3,ACCESORIES_1_158,BOS_3,ACCESORIES,21.488026,5943.0,10.166732,17.0,115.50,132.0,0.25,5.372007,20,29.0
1833,ACCESORIES_1_189_NYC_1,ACCESORIES_1_189,NYC_1,ACCESORIES,0.157063,3070.0,17.580293,29.0,75.00,104.0,0.25,0.039266,20,276.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30321,SUPERMARKET_3_811_BOS_2,SUPERMARKET_3_811,BOS_2,SUPERMARKET,0.459170,6591.0,15.617446,26.0,130.75,156.0,0.15,0.068875,20,276.0
30322,SUPERMARKET_3_811_BOS_3,SUPERMARKET_3_811,BOS_3,SUPERMARKET,0.529053,5257.0,6.677183,11.0,86.00,97.0,0.15,0.079358,20,208.0
30328,SUPERMARKET_3_811_PHI_2,SUPERMARKET_3_811,PHI_2,SUPERMARKET,0.528582,6921.0,8.141825,13.0,163.50,177.0,0.15,0.079287,20,287.0
30329,SUPERMARKET_3_811_PHI_3,SUPERMARKET_3_811,PHI_3,SUPERMARKET,0.528442,3864.0,17.099628,28.0,85.00,113.0,0.15,0.079266,20,207.0


In [157]:
df_final.to_csv("df_final_SS.csv", index=False, encoding="utf-8")